### Train a bilstm to generate jokes in forward and reverse based on a controlled bit

In [1]:
import pickle
import importlib
from library import data_preprocess as dp
importlib.reload(dp)
import random
from time import time

import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM, Bidirectional
from keras.layers import Embedding, TimeDistributed, Flatten, Merge, Concatenate
from keras import regularizers
from keras.metrics import sparse_categorical_accuracy, sparse_categorical_crossentropy
from keras.models import load_model
from keras.optimizers import Adam
from keras.models import Model

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import tensorflow as tf
from keras import backend as K

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Variables

In [2]:
# http://nlp.stanford.edu/data/glove.6B.zip
DATA_PATH = './datasets/jokes.pickle'
VOCAB_PATH = './datasets/jokes_vocabulary.pickle'
MODELS_PATH = './models/'

GLOVE_PATH = './data/glove.6B.200d.txt'

MODEL_PREFIX = 'jokes_controlled_stacked_lstm_glove'

MAX_SEQUENCE_LENGTH = 13
VALIDATION_SPLIT = 0.2

GLOVE_EMBEDDING_DIM = 200
EMBEDDING_DIM1 = 256
EMBEDDING_DIM2 = 256
HIDDEN_DIM1 = 512
HIDDEN_DIM2 = 256
DEEPER_DIM = 256
DROPOUT_FACTOR = 0.2
REGULARIZATION = 0.00001
LEARNING_RATE = 0.003

DATA_PERCENT = 0.1

RUN_INDEX = 1

In [3]:
with open(DATA_PATH, 'rb') as pickleFile:
    sentences = pickle.load(pickleFile)

with open(VOCAB_PATH, 'rb') as pickleFile:
    vocab = pickle.load(pickleFile)
    
random.shuffle(sentences)

print("Number of sentences = ", len(sentences))
print(sentences[:2])
print("Vocab size = ", len(vocab))
print(vocab[:10])

Number of sentences =  96910
['sos sarah jessica parker walks into a bar the bartender asked , " why the long face ? " eos', "sos i'm surprised people still ask me if i want to hold their baby given the number of times i've dropped my phone . eos"]
Vocab size =  8922
['sos', 'did', 'you', 'hear', 'about', 'the', 'new', 'corduroy', 'pillows', '?']


In [4]:
# tokenize data
num_words = len(vocab)

tokenizer = Tokenizer(num_words=None, filters='', lower=True, split=' ', 
                      char_level=False, oov_token=None)
tokenizer.fit_on_texts(sentences)
assert num_words == len(tokenizer.word_index)

encoded_sentences = tokenizer.texts_to_sequences(sentences)
print(encoded_sentences[:5])

VOCAB_SIZE = len(tokenizer.word_index) + 1
print(VOCAB_SIZE)

[[1, 2467, 5058, 5211, 130, 72, 4, 97, 5, 315, 149, 10, 11, 25, 5, 197, 250, 6, 11, 2], [1, 45, 1135, 65, 180, 242, 22, 38, 7, 96, 8, 609, 100, 195, 1683, 5, 510, 19, 585, 153, 1130, 13, 251, 3, 2], [1, 17, 20, 9, 66, 125, 268, 89, 5059, 1055, 6, 5734, 2], [1, 7, 864, 4, 233, 19, 388, 173, 10, 41, 10, 47, 48, 22, 246, 10, 45, 58, 4, 2667, 3, 2], [1, 17, 20, 7, 60, 34, 13, 279, 3775, 4120, 26, 5, 290, 6, 6411, 21, 2]]
8923


In [5]:
# saving
with open(MODELS_PATH + MODEL_PREFIX + '_tokenizer_' + str(RUN_INDEX) + '.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Preparing Training Data

In [21]:
TAG_SIZE = 2
X_data = []
y_data = []
for sentence in encoded_sentences:
    l = len(sentence)
    sliding_window_length = min(l-3, MAX_SEQUENCE_LENGTH)
    step_size = 1
    for i in range(0, l - sliding_window_length, step_size):
        X_data.append(sentence[i:i+sliding_window_length])
        y_data.append(sentence[i+1:i+sliding_window_length+1])
        
print("Total training data size = ", len(X_data))
MAX_SEQ_LEN = max([len(seq) for seq in X_data])
print("Max seq len = ", MAX_SEQ_LEN)

forward_X_data = pad_sequences(X_data, maxlen=MAX_SEQ_LEN, padding='pre')
forward_y_data = pad_sequences(y_data, maxlen=MAX_SEQ_LEN, padding='pre').reshape(-1, MAX_SEQ_LEN, 1)
forward_tag = to_categorical(np.full((forward_X_data.shape[0], MAX_SEQ_LEN), 0), TAG_SIZE)

#print(forward_X_data.shape)
#print(forward_X_data[:2])
#print(forward_y_data.shape)
#print(forward_y_data[:2])
#print(forward_tag.shape)

reverse_X_data = pad_sequences([item[::-1] for item in y_data], maxlen=MAX_SEQ_LEN, padding='pre')
reverse_y_data = pad_sequences([item[::-1] for item in X_data], maxlen=MAX_SEQ_LEN, padding='pre').reshape(-1, MAX_SEQ_LEN, 1)
reverse_tag = to_categorical(np.full((reverse_X_data.shape[0], MAX_SEQ_LEN), 1), TAG_SIZE)

#print(reverse_X_data.shape)
#print(reverse_X_data[:2])
#print(reverse_y_data.shape)
#print(reverse_y_data[:2])
#print(reverse_tag.shape)

X_data = np.concatenate((forward_X_data, reverse_X_data), axis=0)
y_data = np.concatenate((forward_y_data, reverse_y_data), axis=0)
tag_data = np.concatenate((forward_tag, reverse_tag), axis=0)

# shuffle
perm = np.random.permutation(X_data.shape[0])
X_data = X_data[perm]
y_data = y_data[perm]
tag_data = tag_data[perm]
print(X_data.shape)
print(X_data[:4])
print(y_data.shape)
print(y_data[:4])
print(tag_data.shape)
print(tag_data[:4])

Total training data size =  833131
Max seq len =  13
(1666262, 13)
[[ 251   35  479    8   18   52   48    8  150  778   46   10  728]
 [  29  119   27    8  778    3  271   13   33    7   33  132   13]
 [  17   67  334   16   41   10  323   72 2044    4 7407   59    9]
 [  17   90    5 1114   24  776    8 3180   69  333    3   41  116]]
(1666262, 13, 1)
[[[  35]
  [ 479]
  [   8]
  [  18]
  [  52]
  [  48]
  [   8]
  [ 150]
  [ 778]
  [  46]
  [  10]
  [ 728]
  [ 763]]

 [[ 119]
  [  27]
  [   8]
  [ 778]
  [   3]
  [ 271]
  [  13]
  [  33]
  [   7]
  [  33]
  [ 132]
  [  13]
  [  33]]

 [[  67]
  [ 334]
  [  16]
  [  41]
  [  10]
  [ 323]
  [  72]
  [2044]
  [   4]
  [7407]
  [  59]
  [   9]
  [   1]]

 [[  90]
  [   5]
  [1114]
  [  24]
  [ 776]
  [   8]
  [3180]
  [  69]
  [ 333]
  [   3]
  [  41]
  [ 116]
  [  16]]]
(1666262, 13, 2)
[[[0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.

In [7]:
print('Indexing glove word vectors')
#Glove Vectors
glove_embeddings_index = {}
f = open(GLOVE_PATH)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embeddings_index[word] = coefs
f.close()
print('Total %s word vectors.' % len(glove_embeddings_index))

Indexing glove word vectors
Total 400000 word vectors.


In [8]:
print('Preparing glove embedding matrix')
glove_embedding_matrix = np.zeros((VOCAB_SIZE, GLOVE_EMBEDDING_DIM))
for word,i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        glove_embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))
print(glove_embedding_matrix.shape)

Preparing glove embedding matrix
Null word embeddings: 255
(8923, 200)


In [22]:
# define model
def StackedLSTM(vocab_size, glove_embedding_dim, glove_embedding_matrix, embedding_dim1, embedding_dim2,
           hidden_dim1, hidden_dim2, deeper_dim, max_seq_len, tag_size,
           dropout_factor=0.5, regularization=0.00001, learning_rate=0.001):
    
    inputs = Input(shape=(None,))
    tag_inputs = Input(shape=(None,tag_size))
    
    glove_embedding = Embedding(vocab_size, glove_embedding_dim, #input_length=max_seq_len,
                                  weights=[glove_embedding_matrix],
                                  mask_zero=True,trainable=False)(inputs)
    
    word_embedding = Embedding(vocab_size, embedding_dim1, #input_length=max_seq_len, 
                               mask_zero=True, embeddings_regularizer=regularizers.l2(regularization))(inputs)
    
    #tag_embedding = Embedding(tag_size, tag_size, embeddings_regularizer=regularizers.l2(regularization))(tag_inputs)
    
    concat_embeds = Concatenate(axis=-1)([glove_embedding, word_embedding, tag_inputs])
    
    final_embed = Dense(units=embedding_dim2, activation='tanh',
                        kernel_regularizer=regularizers.l2(regularization))(concat_embeds)
    
    lstm1 = LSTM(hidden_dim1, activation='tanh', 
                   kernel_regularizer=regularizers.l2(regularization), 
                   recurrent_regularizer=regularizers.l2(regularization), #unroll=True, 
                   return_sequences = True, dropout=dropout_factor, recurrent_dropout=dropout_factor)(final_embed)
    
    lstm2 = LSTM(hidden_dim2, activation='tanh', 
                   kernel_regularizer=regularizers.l2(regularization), 
                   recurrent_regularizer=regularizers.l2(regularization), #unroll=True, 
                   return_sequences = True, dropout=dropout_factor, recurrent_dropout=dropout_factor)(lstm1)
    
    timedist_dropout = TimeDistributed(Dropout(dropout_factor))(lstm2)
    
    deep_dense = Dense(units=deeper_dim, activation='tanh', 
                       kernel_regularizer=regularizers.l2(regularization))(timedist_dropout)
    
    dropout_layer1 = Dropout(dropout_factor)(deep_dense)
    
    outputs = Dense(units=vocab_size, activation='softmax', 
                    kernel_regularizer=regularizers.l2(regularization))(dropout_layer1)
    
    model = Model(inputs=[inputs,tag_inputs], outputs=outputs)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=learning_rate),
                  metrics=[sparse_categorical_crossentropy, sparse_categorical_accuracy], sample_weight_mode='temporal')
    return model

In [23]:
K.clear_session()
sess = tf.Session()
K.set_session(sess)

model = StackedLSTM(vocab_size=VOCAB_SIZE, glove_embedding_dim=GLOVE_EMBEDDING_DIM,
                    glove_embedding_matrix=glove_embedding_matrix, 
                    embedding_dim1=EMBEDDING_DIM1, embedding_dim2=EMBEDDING_DIM2,
                    hidden_dim1=HIDDEN_DIM1, hidden_dim2=HIDDEN_DIM2,
                    deeper_dim=DEEPER_DIM, max_seq_len=MAX_SEQ_LEN, dropout_factor=DROPOUT_FACTOR, 
                    regularization=REGULARIZATION, learning_rate=LEARNING_RATE, tag_size=TAG_SIZE)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    1784600     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    2284288     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 2)      0                                            
__________________________________________________________________________________________________
concatenat

In [24]:
class TB(TensorBoard):
    def __init__(self, log_every=1, **kwargs):
        super().__init__(**kwargs)
        self.log_every = log_every
        self.counter = 0
    
    def on_batch_end(self, batch, logs=None):
        self.counter+=1
        if self.counter%self.log_every==0:
            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, self.counter)
            self.writer.flush()
        
        super().on_batch_end(batch, logs)

In [ ]:
start_time = time()
tensorboard = TB(log_dir="./logs/" + MODEL_PREFIX + "/{}".format(time()), 
                          histogram_freq=0, write_graph=True, write_images=False, log_every=10)

callbacks=[tensorboard, 
           EarlyStopping(patience=5, monitor='val_loss'),
           ModelCheckpoint(filepath=MODELS_PATH + 'checkpoints/'+ MODEL_PREFIX + '_gen' + str(RUN_INDEX) + '.{epoch:02d}-{val_loss:.2f}.hdf5', 
                           monitor='val_loss', verbose=1, mode='auto', period=1), 
           ModelCheckpoint(filepath=MODELS_PATH + MODEL_PREFIX + '_gen'+str(RUN_INDEX)+'.hdf5', 
                           monitor='val_loss', verbose=1, mode='auto', period=1, save_best_only=True)]

model.fit([X_data, tag_data], y_data, epochs=10, batch_size=1024, shuffle=True, verbose=1, validation_split=0.2, callbacks=callbacks)

print("Total elapsed time: ", time()-start_time)

Train on 1333009 samples, validate on 333253 samples
Epoch 1/10
1333009/1333009 [==============================] - 968s 726us/step - loss: 5.4862 - sparse_categorical_crossentropy: 5.3589 - sparse_categorical_accuracy: 0.1469 - val_loss: 4.5909 - val_sparse_categorical_crossentropy: 4.3533 - val_sparse_categorical_accuracy: 0.2517

Epoch 00001: saving model to ./models/checkpoints/jokes_controlled_stacked_lstm_glove_gen1.01-4.59.hdf5

Epoch 00001: val_loss improved from inf to 4.59085, saving model to ./models/jokes_controlled_stacked_lstm_glove_gen1.hdf5
Epoch 2/10
1246208/1333009 [===========================>..] - ETA: 58s - loss: 4.6709 - sparse_categorical_crossentropy: 4.4015 - sparse_categorical_accuracy: 0.2457

In [68]:
# generate a sequence from a language model
def generate_categorical(model, tokenizer, seed_text, maxlen, probabilistic=False, exploration_factor=1.0, tag=0):
    
    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
    seq = tokenizer.texts_to_sequences([seed_text])[0]
    print(seq)
    
    while True:
        encoded_seq = seq
        if len(seq) > MAX_SEQ_LEN:
            encoded_seq = encoded_seq[-1*MAX_SEQ_LEN:]
            
        #padded_seq = pad_sequences([encoded_seq], maxlen=MAX_SEQ_LEN, padding='pre')
        padded_seq = np.array([seq])
        tags = to_categorical(np.full((1, padded_seq[0].shape[0]), tag), TAG_SIZE)
        y_prob = model.predict([padded_seq,tags])[0][-1].reshape(1,-1)#[3:].reshape(-1,1)
        
        if random.random() <= exploration_factor:
            probabilistic = True
        else:
            probabilistic = False
            
        if probabilistic:
            y_class = np.argmax(np.random.multinomial(1,y_prob[0]/(np.sum(y_prob[0])+1e-5),1))
        else:
            y_class = y_prob.argmax(axis=-1)[0]
        
        if y_class == 0:
            break
        out_word = reverse_word_map[y_class]
        seq.append(y_class)
        if out_word == 'eos' or len(seq) > maxlen or out_word == 'sos':
            break
    
    words = [reverse_word_map[idx] for idx in seq]
    
    return ' '.join(words)

In [69]:
joke = generate(model, tokenizer, "sos i had to use", maxlen=40)
print(joke)

[1, 7, 84, 8, 258]


ValueError: Error when checking : expected embedding_12_input to have shape (10,) but got array with shape (5,)

In [55]:
def bigrams_list(sentence):
    words = sentence.split(' ')
    bigrams = []
    for i in range(0, len(words)-1):
        bigrams.append(words[i]+' '+words[i+1])
    return bigrams

print(bigrams_list("sos hello , i'm a dinosaur . eos"))

['sos hello', 'hello ,', ", i'm", "i'm a", 'a dinosaur', 'dinosaur .', '. eos']


In [56]:
sentence_bigrams = [bigrams_list(s) for s in sentences]
print(sentence_bigrams[:2])

[['sos i', 'i had', 'had to', 'to use', 'use my', 'my glasses', 'glasses when', 'when playing', 'playing tennis', 'tennis .', '. because', 'because its', 'its a', 'a no', 'no contact', 'contact sport', 'sport .', '. eos'], ['sos why', 'why did', 'did the', 'the japanese', 'japanese funeral', 'funeral home', 'home have', 'have to', 'to turn', 'turn away', 'away new', 'new business', 'business ?', '? they', 'they ran', 'ran out', 'out of', 'of san', 'san storage', 'storage eos']]


In [57]:
def intersection(lst1, lst2):
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

def similarity_score(lst1, lst2):
    intersection_len = len(intersection(lst1, lst2))
    return (1.0*intersection_len)/len(lst1)#+len(lst2)-intersection_len)
 
def print_closest_sentences(sentence, sentence_bigrams, top_k=3):
    bigrams = bigrams_list(sentence)
    scores = np.array([similarity_score(bigrams, sbigrams)
                       for sbigrams in sentence_bigrams])
    top_k_indices = scores.argsort()[-1*top_k:][::-1]
    top_k_scores = scores[top_k_indices]
    for k in range(top_k):
        print(top_k_scores[k], " -> ", sentences[top_k_indices[k]])

In [62]:
print_closest_sentences(joke, sentence_bigrams, 10)

0.6363636363636364  ->  sos i made up a new word . plagiarism . eos
0.6363636363636364  ->  sos i just invented a new word . plagiarism . eos
0.6363636363636364  ->  sos i invented a new word . plagiarism . eos
0.6363636363636364  ->  sos i just invented a new joke . i just invented a new word . plagiarism . eos
0.6363636363636364  ->  sos i made a new joke . i made a new word . plagiarism . eos
0.5454545454545454  ->  sos i recently invented a new word to describe a lot of the jokes on the subreddit . plagiarism . eos
0.5454545454545454  ->  sos i invented a new word the other day . plagiarism . eos
0.5454545454545454  ->  sos hey people , i've invented a new word . plagiarism . eos
0.5454545454545454  ->  sos i created a new word today . plagiarism . eos
0.45454545454545453  ->  sos i had to use my glasses when playing tennis . because its a no contact sport . eos


In [65]:
joke = generate(model, tokenizer, "sos what do you call", maxlen=40)
print(joke)
print_closest_sentences(joke, sentence_bigrams, 10)

[1, 17, 20, 9, 66]
sos what do you call a vegetarian ? a rip - off . eos
0.8461538461538461  ->  sos what do you call a cheap circumcision ? a rip - off . eos
0.8461538461538461  ->  sos what do you call a cheap circumcision ? a rip - off . well , you can't blame them . they don't make much money , they just keep the tips . eos
0.8461538461538461  ->  sos what do you call a bad hairdresser who is also very expensive ? a rip - off . eos
0.6923076923076923  ->  sos what do you call a cheap circumcision ? rip - off . eos
0.6923076923076923  ->  sos what do you call a cheap circumcision ? a rip - off ! eos
0.6923076923076923  ->  sos what do you call a gay vegetarian ? a vegetarian . eos
0.6923076923076923  ->  sos what do you call a bad circumcision ? a rip - off eos
0.6923076923076923  ->  sos what do you call a bad circumcision ? what do you call a bad circumcision ? a rip off . eos
0.6923076923076923  ->  sos what do you call a discount circumcision ? a rip off . eos
0.6923076923076923

In [64]:
print(sentences[:10])

['sos i had to use my glasses when playing tennis . because its a no contact sport . eos', 'sos why did the japanese funeral home have to turn away new business ? they ran out of san storage eos', 'sos a world without women would be a pain in the ass . eos', 'sos not everyone that comes into your life needs to stay there . eos', 'sos " i would absolutely say i\'m an introvert ! " - guy screaming to his table full of friends at brunch . eos', 'sos cashier : " would you like to donate to charity today or are you a giant piece of shit ? " eos', "sos what do you call a muslim girl dating an agnostic guy ? for safety purposes , i don't know if i should tell you her name threedots eos", "sos math is so communist threedots threedots there's class struggle for marx eos", "sos i used to be a bodybuilder threedots or ' the dr frankenstein grave robber ' as the press preferred to call me . eos", "sos i can't believe this paper went to college , let alone thought it ruled eos"]
